# Rossman Stors Sales Analysis Using Linear Regression

## Problem Statement / Business Understanding

Rossmann operates over 3,000 drug stores in 7 European countries. Currently, Rossmann store managers are tasked with predicting their daily sales for up to **six weeks** in advance. 

Store sales are influenced by many factors, including promotions, competition, school and state holidays, seasonality, and locality. With thousands of individual managers predicting sales based on their unique circumstances, the accuracy of results can be quite varied.

In this exercise, we are going to **predict 6 weeks of daily sales for 1,115 stores located across Germany**. 

Reliable sales forecasts enable store managers to create effective staff schedules that increase productivity and motivation. By helping Rossmann create a robust **prediction** model, we will help store managers stay focused on what’s most important to them: their customers and their teams!

### Hypothesis Statement

H0: There is no direct correlation between numbers of Rossmann customers and volumes of sales at the stores.

H1: The number of customers influence the volumes of sales at Rossmann stores. The greater the number of customers, the higher the volume of sales. The less the number of customers, the lower the volume of sales.

## Data Understanding

## Data Preparation

## Data Modelling

## Conclusions

## Next Steps